In [ ]:
import flask
from flask import request, jsonify
import joblib
import pandas as pd
import json
import dill
import lime
import lime.lime_tabular
import pickle
import numpy as np

app = flask.Flask(__name__)
app.config["DEBUG"] = False

#Chargement du tableau et du modèle
with open('df_test.pkl', 'rb') as f:
    df = pickle.load(f)
print(df.shape)    
feats = [f for f in df.columns if f not in [
    'TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index']]    
print("feats",len(feats))
df.drop(columns=["index"], inplace=True)
df.set_index("SK_ID_CURR", inplace=True)

#feats = np.genfromtxt('feats.csv', dtype='unicode', delimiter=',')
#print("fets",len(feats))
#feats=[f for f in feats if f not in ['NAME_TYPE_SUITE_Spouse', ' partner', 'WALLSMATERIAL_MODE_Stone', ' brick', 'PREV_NAME_TYPE_SUITE_Spouse', ' partner_MEAN']]
with open('modele_lightgbm.pkl', 'rb') as f:
    model = pickle.load(f)
    print("modéle load ok")


#with open('explainer.pkl', 'rb') as f:
    #explainer = dill.load(f)


def make_prediction(client_id):
    return model.predict_proba([df[feats].loc[client_id]])[0, 1]
    #return model.predict_proba(df[feats].loc[client_id].reshape(1, -1))
    
def explain(client_id):
    print(probleme)
    return explainer.shap_values(df[feats].loc[client_id].to_numpy().reshape(1, -1))[1][0][:]

    #shap_values = explainer.explain_instance(test_df[feats].values[0].reshape(1, -1), model_lgbm.predict_proba).local_exp[0]


@app.route('/', methods=['GET'])
def index():
    return {'message': 'Hello, stranger'}


@app.route('/score_min/', methods=['GET'])
def score_min():
    return {"score_min" : 0.55} 


@app.route('/predict', methods=["GET"])
def proba():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        pred = make_prediction(client_id)
        print(pred)
        return {"proba" : pred}
    else:
        return "Error"

@app.route('/feats/', methods=["GET"])
def feats_ret():
    return json.dumps(feats) 
#.tolist()
@app.route('/importances', methods=["GET"])
def importances():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        shap_vals = explain(client_id).tolist()
        return json.dumps(shap_vals)
    else:
        return "Error"    

@app.route('/bar', methods=["GET"])
def bar():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        feat = str(request.args["feature"])
        
        dff = df[feat]
        retour = []
        retour.append(float(dff.loc[client_id]))
        retour.append(np.mean(dff))
        del dff      
     
        return json.dumps(retour)
    else:
        return "Error"     
@app.route('/boxplot', methods=["GET"])
def boxplot():
    if 'feature' in request.args:
        feat = str(request.args["feature"])
        
        dff = df[feat]
     
        return json.dumps(dff.tolist())
    else:
        return "Error"   
 
if __name__ == "__main__":
    app.run(port=80)

(48744, 798)
feats 795
modéle load ok
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:80
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2023 10:38:50] "GET /feats/ HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:38:51] "GET /score_min/ HTTP/1.1" 200 -
[2023-07-12 10:38:51,230] ERROR in app: Exception on /importances [GET]
Traceback (most recent call last):
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\boudj\AppData\Local\Temp\ipykernel_14612\1539085433.py", line 

0.046085336502454693


127.0.0.1 - - [12/Jul/2023 10:38:51] "GET /boxplot?feature=EXT_SOURCE_3 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:38:51] "GET /bar?client_id=100001&feature=EXT_SOURCE_3 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:01] "GET /feats/ HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:01] "GET /score_min/ HTTP/1.1" 200 -
[2023-07-12 10:41:01,934] ERROR in app: Exception on /importances [GET]
Traceback (most recent call last):
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure

0.046085336502454693


127.0.0.1 - - [12/Jul/2023 10:41:02] "GET /bar?client_id=100001&feature=EXT_SOURCE_3 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:02] "GET /boxplot?feature=EXT_SOURCE_3 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:36] "GET /score_min/ HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:37] "GET /predict?client_id=100001 HTTP/1.1" 200 -
[2023-07-12 10:41:37,019] ERROR in app: Exception on /importances [GET]
Traceback (most recent call last):
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\boudj\anaconda3\envs\env2\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    

0.046085336502454693


127.0.0.1 - - [12/Jul/2023 10:41:37] "GET /boxplot?feature=EXT_SOURCE_3 HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2023 10:41:37] "GET /bar?client_id=100001&feature=EXT_SOURCE_3 HTTP/1.1" 200 -
